+++++++++++++++++++++++++++

This software is available under the terms of the Educational Community License, Version 2.0 (ECL 2.0). This software is Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley").

The text of the ECL license is reproduced below.

Educational Community License, Version 2.0
*************************************
Copyright 2016 The Regents of the University of California, Berkeley ("Berkeley")

Educational Community License, Version 2.0, April 2007

The Educational Community License version 2.0 ("ECL") consists of the
Apache 2.0 license, modified to change the scope of the patent grant in
section 3 to be specific to the needs of the education communities using
this license. The original Apache 2.0 license can be found at:[http://www.apache.org/licenses/LICENSE-2.0]

+++++++++++++++++++++++++++

### This is a utility script used to authenticate the user with CalNet and store the resulting auth and refresh tokens to be used in other scripts.

In [1]:
!source activate py3.5
import os, sys
import boxsdk
print(boxsdk.__version__)
from boxsdk.exception import BoxAPIException

2.0.0a2


In [3]:
!python3 --version
# !conda list


Python 3.5.4 :: Anaconda custom (64-bit)


function to store the oauth2 refresh token in a local file. This can be modified to use a keychain or other as required.

In [15]:
def store_tokens(access_token, refresh_token):
    
    """Callback for storing refresh tokens. (For now we ignore access tokens)."""
    with open('apptoken.cfg', 'w') as f:
        f.write(refresh_token.strip())

In [16]:
def store_client_config(client_id, client_secret, redirect_uri):
    
    """Callback for storing client configuration."""
    with open('app.cfg', 'w') as f:
        f.write(client_id.strip() + '\n')
        f.write(client_secret.strip()+ '\n')
        f.write(redirect_uri.strip()+ '\n')

The client id and client secret are defined in the Box application created for this notebook. Create the application at the Box Developers site: https://berkeley.app.box.com/developers/services/edit/

The redirect uri can be any site that requires validation. Run the bootstrap notebook to create initial 
tokens which are then continually refreshed.

In [17]:
# Define client ID, client secret, and developer token.
# CLIENT_ID = 'ui31zjry6g8lwvkp0zl58qcyp02tsnw4'
# CLIENT_SECRET = 'CgUOz7mCADC1w57lF9XrsTjB2XSy8aM6'
# REDIRECT_URI = 'https://berkeley.account.box.com/login'
DEVELOPER_TOKEN = 'vbrEgLMtkit4yDtDzSSNLD71PIiziD99'
CLIENT_ID = '7fv9qix8gzo77vhwidetl426az41xneg'
CLIENT_SECRET = 'agzSPeOv8Zmmj8iupLJFcl9lWolaBqtY'
REDIRECT_URI ='https://berkeley.app.box.com/folder/0'
store_client_config(CLIENT_ID, CLIENT_SECRET, REDIRECT_URI)

# for debugging
print("DEVELOPER TOKEN: ", DEVELOPER_TOKEN)
print ("CLIENT_ID: ", CLIENT_ID )
print ("CLIENT_SECRET: ", CLIENT_SECRET )
print ("REDIRECT_URI: ", REDIRECT_URI )

DEVELOPER TOKEN:  vbrEgLMtkit4yDtDzSSNLD71PIiziD99
CLIENT_ID:  7fv9qix8gzo77vhwidetl426az41xneg
CLIENT_SECRET:  agzSPeOv8Zmmj8iupLJFcl9lWolaBqtY
REDIRECT_URI:  https://berkeley.app.box.com/folder/0


Perform autentication then create globus client. 
Place the url that is returned into a browser to authenticate with CalNet. CalNet will return a code in the resulting url. 

In [6]:
from boxsdk import OAuth2
oauth = OAuth2(
    client_id=CLIENT_ID.strip(),
    client_secret=CLIENT_SECRET.strip(),
    store_tokens=store_tokens
)

auth_url, csrf_token = oauth.get_authorization_url(REDIRECT_URI.strip())

print ("auth_url", auth_url)

auth_url https://app.box.com/api/oauth2/authorize?state=box_csrf_token_D926dXqt8QHFxxXC&response_type=code&client_id=7fv9qix8gzo77vhwidetl426az41xneg&redirect_uri=https%3A%2F%2Fberkeley.app.box.com%2Ffolder%2F0


Place the code provided in the line below before executing this cell.

__After selecting the Authorise button on the Box login page the following page will have "code=" at the end of the url. Copy the provided code into the following cell instead of clicking the Continue button on that page__

In [7]:
#access_token, refresh_token = oauth.authenticate('paste code here...')
# access_token, refresh_token = oauth.authenticate('lgbyixPBGs7Nw0nK1oTzKDvURuGxVwUS')
# access_token, refresh_token = oauth.authenticate('ui31zjry6g8lwvkp0zl58qcyp02tsnw4')
# access_token, refresh_token = oauth.authenticate('CgUOz7mCADC1w57lF9XrsTjB2XSy8aM6')
# access_token, refresh_token = oauth.authenticate('gwjS5Fj7T59fGJcv')
# access_token, refresh_token = oauth.authenticate('7jQAvcgcKW7iFvlJuhPn7lz23w2f16Rb')
# access_token, refresh_token = oauth.authenticate('ao8lrt3mw2tpestl15xnnivsc7qoi3xt')
# access_token, refresh_token = oauth.authenticate('7xBu4ZDGLTLKj4KU')
access_token, refresh_token = oauth.authenticate('9yGtmCzTnalhfqTpPuIg31hvmal3leG7')
# access_token, refresh_token = oauth.authenticate('sarzJ2qu1MnGKHGgupXisQxzk8ZmlJky')
store_tokens(access_token, refresh_token)
print(refresh_token)

ZLJPGcZz0ENgeJG95me5P3bsJirTJfwT1QhIVyjAD86QWYBBvwdLcJPHrdOz87BR


Create a client to Box and verify that the authorization is valid.
Verify client by retrieving the name of the users root folder in Box.

In [8]:
from boxsdk import Client

client = Client(oauth)

root_folder = client.folder(folder_id='0').get()
print ("folder name: ", root_folder['name'] )

folder name:  All Files
